<a href="https://colab.research.google.com/github/LatoyaHuiZheng/580DHM2/blob/main/grab_web_data(580D_stock).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Write a python script to scrape stock price movement in
# the US stock market on this webpage:
# https://finance.yahoo.com/quote/TSLA/holders

In [3]:
# prompt: Write a python script to scrape stock price movement in the US stock market on this webpage: https://finance.yahoo.com/quote/TSLA/holders

!pip install bs4
!pip install requests

In [4]:
import json
import csv
import sys
from typing import Any, Dict
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
def get_data(ticker_symbol: str) -> Dict[str, Any]:
  """
  Get stock data for a given ticker symbol from Yahoo Finance.
  Parameters:
  - ticker_symbol (Any): Ticker symbol of the stock.
  Returns:
  - Dict[str, Any]: Dictionary containing stock data.
  """
  print('Getting stock data of ', ticker_symbol)

  # Set user agent to avoid detection as a scraper
  headers = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101Firefox/122.0'}
  url = f'https://finance.yahoo.com/quote/{ticker_symbol}'

  r = requests.get(url) #will add timeout here
  soup = BeautifulSoup(r.text, 'html.parser')

  stock = {
    # scraping the stock data from the price indicators
    'symbol': ticker_symbol,
    'stock_name': soup.find('div', {'class':'D(ib) Mt(-5px) Maw(38%)--tab768 Maw(38%) Mend(10px) Ov(h) smartphone_Maw(85%) smartphone_Mend(0px)'}).find_all('div')[0].text.strip(),
    'regularMarketPrice': soup.find('div', {'class':'D(ib) Mend(20px)'}).find_all('fin-streamer')[0].text.strip(),
    'regularMarketChange': soup.find('div', {'class':'D(ib) Mend(20px)'}).find_all('fin-streamer')[1].text.strip(),
    'regularMarketChangePercent': soup.find('div', {'class':'D(ib) Mend(20px)'}).find_all('fin-streamer')[2].text.strip(),
    'quote': soup.find('div', {'class':'D(ib) Mend(20px)'}).find_all('span')[2].text.strip(),

    # scraping the stock data from the "Summary" table
    'previous_close': soup.find('table', {'class':'W(100%)'}).find_all('td')[1].text.strip(),
    'open_value': soup.find('table', {'class':'W(100%)'}).find_all('td')[3].text.strip(),
    'bid': soup.find('table', {'class':'W(100%)'}).find_all('td')[5].text.strip(),
    'ask': soup.find('table', {'class':'W(100%)'}).find_all('td')[7].text.strip(),
    'days_range': soup.find('table', {'class':'W(100%)'}).find_all('td')[9].text.strip(),
    'week_range': soup.find('table', {'class':'W(100%)'}).find_all('td')[11].text.strip(),
    'volume': soup.find('table', {'class':'W(100%)'}).find_all('td')[13].text.strip(),
    'avg_volume': soup.find('table', {'class':'W(100%)'}).find_all('td')[5].text.strip(),
    'market_cap': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[1].text.strip(),
    'beta': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[3].text.strip(),
    'pe_ratio': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[5].text.strip(),
    'eps': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[7].text.strip(),
    'earnings_date': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[9].text.strip(),
    'dividend_yield': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[11].text.strip(),
    'ex_dividend_date': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[13].text.strip(),
    'year_target_est': soup.find('table', {'class':'W(100%) M(0) Bdcl(c)'}).find_all('td')[15].text.strip(),
  }
  return stock

In [6]:
get_data('TSLA')

Getting stock data of  TSLA


{'symbol': 'TSLA',
 'stock_name': 'Tesla, Inc. (TSLA)',
 'regularMarketPrice': '197.41',
 'regularMarketChange': '+2.64',
 'regularMarketChangePercent': '(+1.36%)',
 'quote': 'At close:  04:00PM EST',
 'previous_close': '194.77',
 'open_value': '194.00',
 'bid': '197.29 x 900',
 'ask': '196.79 x 1200',
 'days_range': '191.36 - 198.32',
 'week_range': '152.37 - 299.29',
 'volume': '92,081,183',
 'avg_volume': '197.29 x 900',
 'market_cap': '628.709B',
 'beta': '2.43',
 'pe_ratio': '45.80',
 'eps': '4.31',
 'earnings_date': 'Apr 17, 2024 - Apr 22, 2024',
 'dividend_yield': 'N/A (N/A)',
 'ex_dividend_date': 'N/A',
 'year_target_est': '207.75'}

In [7]:
def get_holders(ticker_symbol: str) -> Dict[str, Any]:
  headers = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101Firefox/122.0'}
  url = f'https://finance.yahoo.com/quote/{ticker_symbol}/holders'

  r = requests.get(url, headers=headers) #will add timeout here

  holders_info = {
    'symbol': ticker_symbol}

  summary_df, institution_df, mutual_fund_df = tokens = pd.read_html(r.text)
  for i, (number, holder_name) in enumerate(summary_df.values):
    holder_name = holder_name.split('% ')[-1]
    holders_info[f'MajorHolders{i + 1}'] = f'{number} {holder_name}'

  for i, (holder, share) in enumerate(institution_df[['Holder', '% Out']].values):
    holders_info[f'TopInstitutionalHolder{i + 1}'] = f'{holder} {share}'

  for i, (holder, share) in enumerate(mutual_fund_df[['Holder', '% Out']].values):
    holders_info[f'TopMutualFundHolder{i + 1}'] = f'{holder} {share}'
  return holders_info

In [8]:
def get_profile(ticker_symbol: str) -> Dict[str, Any]:
  headers = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101Firefox/122.0'}
  url = f'https://finance.yahoo.com/quote/{ticker_symbol}/profile'

  r = requests.get(url, headers=headers) #will add timeout here
  executive_df = pd.read_html(r.text)[0]
  soup = BeautifulSoup(r.text, 'html.parser')
  profile_info = {
      'symbol': ticker_symbol,
      'StockName': soup.find('h1', {"class": "D(ib) Fz(18px)"}).text.strip(),
      'Address': '; '.join([x.text for x in soup.find('p', {'class':'D(ib) W(47.727%) Pend(40px)'}).children]),
      'Description': soup.find('p', {'class': 'Mt(15px) Lh(1.6)'}).text.strip()
  }
  for i, name in enumerate(executive_df['Name'].tolist()):
    profile_info[f'Executive{i+1}'] = name
  return profile_info

In [9]:
get_holders('META')

{'symbol': 'META',
 'MajorHolders1': '0.17% of Shares Held by All Insider',
 'MajorHolders2': '79.60% of Shares Held by Institutions',
 'MajorHolders3': '79.73% of Float Held by Institutions',
 'MajorHolders4': '4664 Number of Institutions Holding Shares',
 'TopInstitutionalHolder1': 'Vanguard Group Inc 8.47%',
 'TopInstitutionalHolder2': 'Blackrock Inc. 7.16%',
 'TopInstitutionalHolder3': 'FMR, LLC 6.08%',
 'TopInstitutionalHolder4': 'State Street Corporation 4.05%',
 'TopInstitutionalHolder5': 'JP Morgan Chase & Company 2.23%',
 'TopInstitutionalHolder6': 'Geode Capital Management, LLC 2.09%',
 'TopInstitutionalHolder7': 'Price (T.Rowe) Associates Inc 2.07%',
 'TopInstitutionalHolder8': 'Capital World Investors 1.73%',
 'TopInstitutionalHolder9': 'Morgan Stanley 1.65%',
 'TopInstitutionalHolder10': 'Capital International Investors 1.58%',
 'TopMutualFundHolder1': 'Vanguard Total Stock Market Index Fund 3.15%',
 'TopMutualFundHolder2': 'Vanguard 500 Index Fund 2.43%',
 'TopMutualFundH

In [10]:
get_profile('META')

{'symbol': 'META',
 'StockName': 'Meta Platforms, Inc. (META)',
 'Address': '1 Meta Way; ; Menlo Park, CA 94025; ; United States; ; 650 543 4800; ; https://investor.fb.com',
 'Description': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, and wearables worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment offers Facebook, which enables people to share, discuss, discover, and connect with interests; Instagram, a community for sharing photos, videos, and private messages, as well as feed, stories, reels, video, live, and shops; Messenger, a messaging application for people to connect with friends, family, communities, and businesses across platforms and devices through text, audio, and video calls; and WhatsApp, a messaging application that is used by people and businesses to communicate and transac

In [11]:
get_holders('TSLA')

{'symbol': 'TSLA',
 'MajorHolders1': '13.00% of Shares Held by All Insider',
 'MajorHolders2': '45.74% of Shares Held by Institutions',
 'MajorHolders3': '52.58% of Float Held by Institutions',
 'MajorHolders4': '3863 Number of Institutions Holding Shares',
 'TopInstitutionalHolder1': 'Vanguard Group Inc 7.22%',
 'TopInstitutionalHolder2': 'Blackrock Inc. 5.91%',
 'TopInstitutionalHolder3': 'State Street Corporation 3.42%',
 'TopInstitutionalHolder4': 'Geode Capital Management, LLC 1.74%',
 'TopInstitutionalHolder5': 'Capital World Investors 1.29%',
 'TopInstitutionalHolder6': 'Morgan Stanley 1.04%',
 'TopInstitutionalHolder7': 'Norges Bank Investment Management 0.99%',
 'TopInstitutionalHolder8': 'FMR, LLC 0.92%',
 'TopInstitutionalHolder9': 'JP Morgan Chase & Company 0.87%',
 'TopInstitutionalHolder10': 'Price (T.Rowe) Associates Inc 0.82%',
 'TopMutualFundHolder1': 'Vanguard Total Stock Market Index Fund 2.64%',
 'TopMutualFundHolder2': 'Vanguard 500 Index Fund 2.08%',
 'TopMutualFu

In [12]:
get_holders('AAPL')

{'symbol': 'AAPL',
 'MajorHolders1': '5.94% of Shares Held by All Insider',
 'MajorHolders2': '56.68% of Shares Held by Institutions',
 'MajorHolders3': '60.25% of Float Held by Institutions',
 'MajorHolders4': '6149 Number of Institutions Holding Shares',
 'TopInstitutionalHolder1': 'Vanguard Group Inc 8.54%',
 'TopInstitutionalHolder2': 'Blackrock Inc. 6.75%',
 'TopInstitutionalHolder3': 'Berkshire Hathaway, Inc 5.86%',
 'TopInstitutionalHolder4': 'State Street Corporation 3.80%',
 'TopInstitutionalHolder5': 'Geode Capital Management, LLC 1.95%',
 'TopInstitutionalHolder6': 'FMR, LLC 1.94%',
 'TopInstitutionalHolder7': 'Morgan Stanley 1.41%',
 'TopInstitutionalHolder8': 'Price (T.Rowe) Associates Inc 1.37%',
 'TopInstitutionalHolder9': 'Norges Bank Investment Management 1.14%',
 'TopInstitutionalHolder10': 'Northern Trust Corporation 1.05%',
 'TopMutualFundHolder1': 'Vanguard Total Stock Market Index Fund 3.00%',
 'TopMutualFundHolder2': 'Vanguard 500 Index Fund 2.29%',
 'TopMutualFu

In [34]:
holders_info1 = get_holders('TSLA')
holders_info2 = get_holders('AMZN')
holders_info3 = get_holders('GOOG')
# Assume df1, df2 and df3 are your data frames
df1 = pd.DataFrame.from_dict(holders_info1, orient='index').T
df2 = pd.DataFrame.from_dict(holders_info2, orient='index').T
df3 = pd.DataFrame.from_dict(holders_info3, orient='index').T

# Merge data frames
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Save the merged data frame as a CSV file
combined_df.to_csv('combined_data.csv', index=False)

# save holders data to  Excel file
combined_df.to_excel('combined_data.xlsx', index=False)

# save holders data to JSON file
combined_dict = combined_df.to_dict()
with open('combined_dict.json', 'w', encoding='utf-8') as f:
    json.dump(combined_dict, f)

# download to location
from google.colab import files
files.download('combined_data.csv')
files.download('holders_info.json')
files.download('combined_data.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>